In [1]:
## Below is Sage code given in the project which computes the Picard group for the Fermat cubic x^3 + y^3 + z^3 = 1

from itertools import combinations
from itertools import chain
from collections import Counter

def is_perm(items0, items1): #Determines whether two tuples/lists are permutations of each other
  return len(items0) == len(items1) and Counter(items0) == Counter(items1)
    

k = CyclotomicField(3) #field Q[ζ] where ζ = exp(2πi/3); using CC doesn't determine all intersections of lines below
a = k.gen() #corresponds to exp(2πi/3)
P.<x,y,z,w> = ProjectiveSpace(3,k) 

G = FreeGroup(27) #Free group generated by 27 lines

# Three family of lines from previous proposition
l_1 = [(x + y*a^i, w + z*a^j) for i in [0,1,2] for j in [0,1,2]]
l_2 = [(z + x*a^i, w + y*a^j) for i in [0,1,2] for j in [0,1,2]]
l_3 = [(z + y*a^i, w + x*a^j) for i in [0,1,2] for j in [0,1,2]] 
lines = l_1 + l_2 + l_3
lines_x = list(G.generators()) #Symbolic representation of lines ([x0, ..., x26]) - groups relations are given in terms of these symbols

# Pairs of lines 
itr = combinations(lines, 2) 
list1 = []
for i in itr:
    flattened = [item for sublist in i for item in sublist] #flatten pairs of tuples - necessary for computing P.subscheme below
    list1 += [flattened]
    
list_x = [x for x in combinations(lines_x, 2)] #Symbolic pairs of lines e.g (x0, x22)


# Determine whether two lines intersect or not 
l = []
a_x = []
for i in range(len(list1)):
    X = P.subscheme(list1[i]) #P is our projective 3-space over k, computes a subvariety defined by equations in 
                              #list of lines 
    if X.dimension() == 0: #If two lines intersect it is at a point, which has dimension 0
        l += [[list1[i]]] #List of pairs of (actual) lines which intersect
        a_x += [list_x[i]] #List of pairs of (symbolic) lines which intersect
print('The number of pairs of lines which intersect is:', len(a_x))

s_x = [(a_x[i][1], a_x[i][0]) for i in range(len(a_x))]
l_x = sorted(a_x + s_x) #List of pairs of (symbolic) lines which intersect and their transpositions e.g. 
                        #both (x0, x1) and (x1, x0) are both in this list - necessary for algorithm below
    
#list of triangles
# We look for intersecting pairs of lines which form a triangle i.e we have (x_i, x_j), (x_j, x_k), (x_k, x_i)
# which form the triangle (x_i, x_j, x_k)

tris = []
for i in range(len(l_x)):
    l1 = l_x[i]
    for j in range(i+1, len(l_x)):
        l2 = l_x[j] 
        if l2[0] != l1[1]:
            continue
        #print(l1, l2)
        for k in range(j+1, len(l_x)):
            l3 = l_x[k]  #l1, l2, l3 are in ascending order 
            if l2[1] != l3[0]:
                continue
            if l3[1] == l1[0]:
                tri = tuple(set(l1+l2+l3)) #triangle (x_i, x_j, x_k)
                tris += [tri]
                break
print('The number of triangles we have computed is:', len(tris))

# Relations on our free group G of 27 lines the first ensures the group is abelian and the second 'triangle = constant' as stated
# in Miles Reid [INSERT REFERENCE]
abelian = [lines_x[i]*lines_x[j]*lines_x[i]^(-1)*lines_x[j]^(-1) for i in range(len(lines_x)) for j in range(len(lines_x))]
tri_const = [tris[i][0]*tris[i][1]*tris[i][2]*tris[j][0]^(-1)*tris[j][1]^(-1)*tris[j][2]^(-1) for i in range(len(tris)) for j in range(len(tris))]
relations = abelian + tri_const

H = (G/ relations).simplified() 

print('Our group is the', H)

The number of pairs of lines which intersect is: 135
The number of triangles we have computed is: 45
Our group is the Finitely presented group < x2, x3, x8, x9, x17, x21, x24 | x9^-1*x21*x9*x21^-1, x9^-1*x3*x9*x3^-1, x21^-1*x24*x21*x24^-1, x21^-1*x17^-1*x21*x17, x9*x17*x9^-1*x17^-1, x9*x2*x9^-1*x2^-1, x8^-1*x24^-1*x8*x24, x8^-1*x21^-1*x8*x21, x2*x24^-1*x2^-1*x24, x8^-1*x17*x8*x17^-1, x17^-1*x3*x17*x3^-1, x3^-1*x21*x3*x21^-1, x17^-1*x2*x17*x2^-1, x2*x21^-1*x2^-1*x21, x24^-1*x9*x24*x9^-1, x8*x9*x8^-1*x9^-1, x24*x3^-1*x24^-1*x3, x3^-1*x2*x3*x2^-1, x17*x24^-1*x17^-1*x24, x3*x8*x3^-1*x8^-1, x2^-1*x8^-1*x2*x8, x3*x24^-1*x17*x8^-1*x24*x3^-1*x8*x17^-1, x17*x8^-1*x24*x3*x17^-1*x21^-1*x8*x3^-1*x24^-1*x21, x9*x21^-1*x2^-1*x17^-1*x3*x9^-1*x21*x17*x2*x3^-1, x9^-1*x2*x8*x9^-1*x3*x21*x2^-1*x9*x8^-1*x9*x21^-1*x3^-1 >
